### Imports modules and libs

In [1]:
import pandas as pd 
import numpy as np
from ConvertFile import Convert_File_to_pd
from DataAnalyze import DataAnalyze
from AnalyzeDuplicatedData import AnalyzeDuplicatedData
from DataTransform import DataTransform
from LoadCsvs import LoadCsvs


### Transform CSVs to DFs

In [2]:
Data_frame=Convert_File_to_pd(file_name='Team',name_table='Data',folder_name='ETL-Data-FIFA')

In [3]:
Df=Data_frame.convert_to_dataframe

### Exploratory analysis to investigate the possibility of data cleaning

In [4]:
#Imprimindo o DataFrame
Df

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [5]:
#Classe responsável pelas análises exploratórias
Results_analysis=DataAnalyze(Df)

#### Detect NaNs

In [6]:
# No objetivo de não ferir a integridade de domínio quando estes dados forem carregados em um bd, o id não pode ter campos nulos ou NaN
Results_analysis.detect_nans(name_column='id')


'lista  indices contendo nans: []'

In [7]:
#Fazend isso para player_api_id 
Results_analysis.detect_nans(name_column='team_api_id')

'lista  indices contendo nans: []'

In [8]:
#Fazend isso para player_fifa_api_id	
Results_analysis.detect_nans(name_column='team_fifa_api_id')

'lista  indices contendo nans: [8, 14, 170, 204, 208, 213, 223, 225, 232, 233, 298]'

#### Detect Data Duplicated

In [9]:
#além disso, é necessário avaliar os registros duplicados quando a coluna não pode contê-los, como é o caso das colunas acima 
Result_dup_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='id')
Result_dup_team_api_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='team_api_id')
Result_dup_team_fifa_api_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='team_fifa_api_id')

In [10]:
Result_dup_id.print_duplicated_data(table_name='Player')
Result_dup_team_api_id.print_duplicated_data(table_name='Player')
Result_dup_team_fifa_api_id.print_duplicated_data(table_name='Player')

a quantidade de dados duplicados da tabela Player  na coluna id são 0
a quantidade de dados duplicados da tabela Player  na coluna team_api_id são 0
a quantidade de dados duplicados da tabela Player  na coluna team_fifa_api_id são 13


In [11]:
Result_dup_team_fifa_api_id.detect_values_data()

[nan, 111560.0, nan, 111429.0, 301.0, nan, nan, nan, nan, nan, nan, nan, nan]

#### Identify Outliers

In [12]:
#Como não existem colunas quantitativas, não é necessário avaliação de outliers


### Analyze  typing

In [13]:
#analisando a tipagem esperada de forma geral
Df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                299 non-null    int64  
 1   team_api_id       299 non-null    int64  
 2   team_fifa_api_id  288 non-null    float64
 3   team_long_name    299 non-null    object 
 4   team_short_name   299 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


In [14]:
#Como ids sob tipagem numérica são geralmente representados por inteiros,logo é necessário avaliar a coluna team_fifa_api_id por estar como float64

Results_analysis.analyze_typing_expected_number(expected_type_number='int',name_column='team_fifa_api_id')


{0: [673.0, 'same expected value between int and float typing'],
 1: [675.0, 'same expected value between int and float typing'],
 2: [15005.0, 'same expected value between int and float typing'],
 3: [2007.0, 'same expected value between int and float typing'],
 4: [1750.0, 'same expected value between int and float typing'],
 5: [229.0, 'same expected value between int and float typing'],
 6: [674.0, 'same expected value between int and float typing'],
 7: [1747.0, 'same expected value between int and float typing'],
 9: [232.0, 'same expected value between int and float typing'],
 10: [110724.0, 'same expected value between int and float typing'],
 11: [231.0, 'same expected value between int and float typing'],
 12: [546.0, 'same expected value between int and float typing'],
 13: [100081.0, 'same expected value between int and float typing'],
 15: [111560.0, 'same expected value between int and float typing'],
 16: [681.0, 'same expected value between int and float typing'],
 17: 

In [15]:
#avaliando se todos os dados seguem com a mesma quantidade de validações entre float e int, considerando também os NaNs

cont_float_and_int=len(Results_analysis.analyze_typing_expected_number(expected_type_number='int',name_column='team_fifa_api_id'))
print(cont_float_and_int+Df['team_fifa_api_id'].isna().sum()==(Df['team_fifa_api_id'].shape[0]))

True


In [16]:
#confirmando se todos os dados estão vindo como str para as colunas team_long_name e team_short_name

Results_analysis.confirm_typing(name_column='team_long_name',type_expected='str')


[]

In [17]:
Results_analysis.confirm_typing(name_column='team_short_name',type_expected='str')

##Como as listas vieram vazias, todos os dados estão coerentes como str

[]

### Data transform 

In [18]:
#Instânciando o Df para a classe DataCleaning

Data_cleaning=DataTransform(dataframe_to_transform=Df)

#### Remove(or not) NaNs

In [19]:
# partindo do pressuposto de que as colunas identificadoras (id, team_api_id e team_fifa_api_id)  não podem ter valores NaNs, é preciso remove-los
#Por possuirem valores distintos e não apresentam uma progressão lógica neste caso, além de serem propriamente identificadores

#Logo, assumiremos a remoção, haja vista que a proporção de NaNs próximos ou abaixo de 5 %  servirá como parâmetro para remoção 
Results_analysis.count_nans(name_columns=['id','team_api_id','team_fifa_api_id'])

{'id': 0, 'team_api_id': 0, 'team_fifa_api_id': 3.678929765886288}

In [20]:
#Removendo os NaNs dos team_fifa_api_id

Data_cleaning.remove_nans(name_columns=['team_fifa_api_id'])

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
283,49124,10243,894.0,FC Zürich,ZUR
284,49479,10190,898.0,FC St. Gallen,GAL
285,49837,10191,1715.0,FC Thun,THU
286,50201,9777,324.0,Servette FC,SER


#### Remove(or not) Duplicated


In [21]:
#removendo os dados duplicados de team_fifa_api_id

Data_Transf=Data_cleaning.remove_dup(name_columns=['team_fifa_api_id'])

#### Data Typing 

In [22]:
#Transformando a coluna Team_fifa_api_id em int
Data_Transf=Data_cleaning.transf_typing(name_columns=['team_fifa_api_id'],type_expected='int')

### Load CSV

In [23]:
Df_load=LoadCsvs(dataframe=Data_Transf,new_folder='Load_csvs',name_dataframe='Team',folder_name=Data_frame.folder_name)

In [24]:
Df_load.load_csv